In [7]:
import os
import pandas as pd 
from tqdm import tqdm
import re
import codecs

In [1]:
def find_total_matches(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
        occurrences = 0
        occurrences += text.count("purchase agreement")
        #occurrences += text.count("purchase contract")
        return occurrences

In [13]:
def generate_directories(start_year, end_year):
    directories = []
    quarters = [1,2,3,4]
    for year in range(start_year, end_year + 1):
        for quarter in quarters:
            directory = f"sec-data/data/{year}_{quarter}/10-K"
            directories.append(directory)
    return directories


In [2]:
def find_all(a_str, sub):
    start = 0
    while True:
        start = a_str.find(sub, start)
        if start == -1: return
        yield start
        start += len(sub)
        
list(find_all('spam spam spam     spam', 'spam'))

[0, 5, 10, 19]

In [12]:
def find_purchase_contract_context(file_path):
    pattern = re.compile(r'([^\.]*?purchase agreement[^\.]*\.)', re.IGNORECASE)

    # Initialize list to store contexts
    contexts = []

    with codecs.open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
        text = file.read()

        instances = list(find_all(text, 'purchase agreement'))
        if len(instances) == 0:
            return contexts

        # Find matches in the text
        for instance in instances:
            start_index = max(0, instance - 500)
            end_index = min(len(text), instance + 500)
            context = text[start_index:end_index]
            contexts.append(context)

    return contexts

In [9]:
def find_purchase_all_context(file_path):

    # Initialize list to store contexts
    contexts = []

    with codecs.open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
        text = file.read()

        instances = list(find_all(text, 'purchase contract'))
        if len(instances) == 0:
            return contexts

        '''
        instances += list(find_all(text, 'purchase agreement'))

        if len(instances) < 5:
            return contexts
        '''
        
        # Find matches in the text
        for instance in instances:
            start_index = max(0, instance - 300)
            end_index = min(len(text), instance + 300)
            context = text[start_index:end_index]
            contexts.append(context)

    return contexts

In [10]:
file = "sec-data/data/1995_1/10-K/0000893220-95-000104.txt"
find_purchase_all_context(file)

["C2-85-1209, (U. S. Dist. Ct., S. D. Ohio, filed July 26,\n1985).  See II.C.1. below\n\n          2.   Phillips Production Co. v. Columbia Gas Transmission Corp., C.A.\nNo. 89-0269, (U.S. Dist. Ct., W.D. Pa. filed February 7, 1989).  The complaint\nas filed contained six separate counts involving ten gas purchase contracts\nwith Columbia Transmission.  All claims except those relating to Columbia\nTransmission's invocation of the cost recovery clause were settled and\ndismissed December 18, 1989, pursuant to agreement of the parties.  Phillips\ncost recovery claim was stayed by Columbia Transmission's b",
 'are stayed as to Columbia Transmission; indemnification agreements will be\neffective if the contract providing indemnification is not rejected)\n\n          1.  Royalty Owners Litigation: The agreements between Columbia\nTransmission and certain southwest producers effective in 1985 which reformed\ngas purchase contracts have resulted in a number of lawsuits against the\nproducers

In [14]:

start_year = 1995
end_year = 1995
directories = generate_directories(start_year, end_year)


csv_path = "combined.csv"
if os.path.exists(csv_path):
    existing_df = pd.read_csv(csv_path)
else:
    existing_df = pd.DataFrame()

data = []
for directory in directories:
    print(directory)
    file_list = os.listdir(directory)

    for filename in tqdm(file_list):
        file_path = os.path.join(directory, filename)
        
        if file_path.endswith(".txt"):

            contexts = find_purchase_contract_context(file_path)
            
            if len(contexts) != 0:
                for paragraph in contexts:
                    data.append({'Directory': directory, 'File': filename, 'Paragraph': paragraph})

df = pd.DataFrame(data)

combined_df = pd.concat([existing_df, df], ignore_index=True)
combined_df.to_csv(csv_path, index=True)


sec-data/data/1995_1/10-K


100%|██████████| 1281/1281 [00:14<00:00, 87.11it/s]


sec-data/data/1995_2/10-K


100%|██████████| 253/253 [00:02<00:00, 104.05it/s]


sec-data/data/1995_3/10-K


100%|██████████| 305/305 [00:03<00:00, 84.85it/s] 


sec-data/data/1995_4/10-K


100%|██████████| 339/339 [00:04<00:00, 77.05it/s] 


In [15]:

data = []

for directory in directories:
    counter =0

    file_list = os.listdir(directory)

    for filename in tqdm(file_list):
        file_path = os.path.join(directory, filename)
        
        # Check if the file is a text file
        if file_path.endswith(".txt"):
            # Find instances of "purchase contract" in the file
            occurrences = find_purchase_contract_context(file_path)
            
            # Print the filename and the number of occurrences
            if occurrences != 0:
                print(f"File: {filename}, Occurrences: {occurrences}")
                counter += 1
                
    data.append({'Directory': directory, 'Purchase Contract Mentioned': counter, 'Total Files': len(file_list)})

df = pd.DataFrame(data)
df.to_csv("count_purchase_contract.csv", index=False)


  0%|          | 0/1281 [00:00<?, ?it/s]

 17%|█▋        | 215/1281 [00:00<00:00, 1085.00it/s]

File: 0000912057-95-000305.txt, Occurrences: []
File: 0000100122-95-000007.txt, Occurrences: []
File: 0000100166-95-000031.txt, Occurrences: ['uch bank being an "Approved Bank"), in each case with\nmaturities of not more than 270 days from the date of\nacquisition, (iii) commercial paper and variable or fixed rate\nnotes issued by any Approved Bank (or by the parent company\nthereof) or any variable rate notes issued by, or guaranteed by\nany domestic corporation rated A-2 (or the equivalent thereof)\nor better by S&P or P-2 (or the equivalent thereof) or better by\nMoody\'s and maturing within six months of the date of\nacquisition and (iv) repurchase agreements with a bank or trust\ncompany (including the Bank) or recognized securities dealer\nhaving capital and surplus in excess of $500,000,000 for direct\nobligations issued by or fully guaranteed by the United States\nof America in which the Borrower shall have a perfected first\npriority security interest (subject to no other lien

 25%|██▌       | 324/1281 [00:00<00:00, 1046.86it/s]

File: 0000029917-95-000005.txt, Occurrences: []
File: 0000029924-95-000002.txt, Occurrences: []
File: 0000891092-95-000025.txt, Occurrences: []
File: 0000003000-95-000003.txt, Occurrences: ["pany has available $30,000,000 of financing under uncommitted\nmoney market lines of credit with several banks.  These facilities bear\ninterest at rates that vary with the banks' cost of funds and are typically\nless than the prevailing bank prime rate.  These credit lines are used in\nconjunction with the revolving credit agreement to facilitate settlement\nand accommodate short-term borrowing fluctuations.\n     Note payable to vendor consists of a non-interest bearing financing\nprovided through a purchase agreement with a vendor for the acquisition of\nan aircraft.\n     The Company has classified the borrowings outstanding under the money\nmarket lines of credit and note payable to vendor as long-term.  These\namounts will be refinanced under the revolving credit agreement.\n     The Company'

...

 56%|█████▌    | 190/339 [00:00<00:00, 865.30it/s]

File: 0000912057-95-011125.txt, Occurrences: []
File: 0000074046-95-000013.txt, Occurrences: []
File: 0000744106-95-000012.txt, Occurrences: []
File: 0000950134-95-002574.txt, Occurrences: []
File: 0000745026-95-000026.txt, Occurrences: []
File: 0000889812-95-000784.txt, Occurrences: ["roperties as its primary source of\nliquidity. During the year ended September 30, 1995, the Radisson South\ngenerated positive cash flow while the Somerset Marriott Hotel generated\nnegative cash flow due to significant property improvements. The Holiday Inn\nCrowne Plaza, owned by the unconsolidated joint venture, experienced positive\ncash flow during the year ended September 30, 1995. Working capital reserves are\nusually invested in United States Treasury obligations, money market accounts\nand repurchase agreements secured by United States Treasury obligations.\n\n     Registrant distributed $32,334,000 to the limited partners ($354.99 per\nlimited partnership unit) and $660,000 to the General Part

 82%|████████▏ | 277/339 [00:00<00:00, 768.67it/s]

File: 0000008154-95-000035.txt, Occurrences: []
File: 0000914233-95-000037.txt, Occurrences: []
File: 0000950115-95-000456.txt, Occurrences: []
File: 0000081870-95-000075.txt, Occurrences: []
File: 0000892569-95-000741.txt, Occurrences: []
File: 0000930661-95-000501.txt, Occurrences: []
File: 0000819539-95-000017.txt, Occurrences: []
File: 0000909654-95-000102.txt, Occurrences: ["empts\n         to  manage  this  risk  and  utilizes   off-balance   sheet   financial\n         instruments to a limited extent to manage its risks.\n\n         Home Federal obtains advances from the FHLB-NY upon the security of its\n         residential  mortgage  loans  and  mortgage-backed   securities.   Such\n         advances are made pursuant to several  different credit programs,  each\n         of which has its own interest rate and range of maturities.\n\n         Home Federal also employs repurchase agreements as a means of borrowing\n         funds.  It is the Savings Bank's policy to enter into 

100%|██████████| 339/339 [00:00<00:00, 813.64it/s]
